## Train Texts preparing
 Main + test

In [1]:
ls -lh ../data/main/

total 12K
drwxr-xr-x 2 fogside fogside 4,0K янв 14 17:04 active-dict/
drwxr-xr-x 2 fogside fogside 4,0K янв 14 17:03 bts-rnc/
drwxr-xr-x 2 fogside fogside 4,0K янв 13 20:52 wiki-wiki/


In [3]:
ls -lh ../data/my_data/

total 3,4G
-rw-r--r-- 1 fogside fogside   11M янв 15 15:21 dict.txt
-rw-r--r-- 1 fogside fogside  620M янв 14 19:34 main_words_wiki_normalized_no_punct.txt
-rw-r--r-- 1 fogside fogside  732M янв 14 18:56 main_words_wiki.txt
-rw-r--r-- 1 fogside fogside 1019M окт 20 00:10 ruwiki_00.txt
-rw-r--r-- 1 fogside fogside  1,1G янв 13 15:06 ruwiki_tokenized.txt


In [4]:
import pandas as pd
import os

In [5]:
lol = pd.read_csv("../data/main/active-dict/test.csv", sep='\t')
lol.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,2074,давление,NaN,NaN,0-9,Давление пара создается движением поршня в цил...
1,2075,давление,NaN,NaN,13-22,"«У тебя что, давление поднялось?» Я сказал, чт..."
2,2076,давление,NaN,NaN,56-65,Я жалуюсь Никоновичу наконец на головокружение...
3,2077,давление,NaN,NaN,0-9,Давление в котле не менялось
4,2078,давление,NaN,NaN,25-34,Он каждые два часа мерил давление и сахар в крови


In [13]:
list(lol.context.to_dense())

['Давление пара создается движением поршня в цилиндре',
 '«У тебя что, давление поднялось?» Я сказал, что не мерил, но, наверное, поднялось',
 'Я жалуюсь Никоновичу наконец на головокружение и низкое давление, и лучше бы я этого не делал',
 'Давление в котле не менялось',
 'Он каждые два часа мерил давление и сахар в крови',
 'Узор, образованный гранатами, […] свидетельствует о том, что они выделились из железо-никелевого сплава в твердом состоянии, а это могло произойти только в результате ослабления давления на вещество, поднимавшееся из глубин Земли',
 'обработка металла давлением',
 'На директора оказывали грубое давление',
 'На моего подзащитного оказывается психологическое давление',
 'Из-за неравномерного давления снега на стенки палатка перекосилась',
 'Подобный взгляд […], вероятно, отражает давление аграрной среды на церковного писателя',
 'Караваев ненавидел советскую власть и отважно противостоял ее давлению',
 'Колонка находилась рядом – на другой стороне улицы, но давлени

In [17]:
path = "../data/main/"
train_contexts = []
test_contexts = []

for (dirpath, dirnames, filenames) in os.walk(path):
    for dirname in dirnames:
        for (dirpath1, _, filenames1) in os.walk(os.path.join(path, dirname)):
            print(os.path.join(path, dirname))
            df = pd.read_csv(os.path.join(path, dirname, "train.csv"), sep='\t')
            df_tst = pd.read_csv(os.path.join(path, dirname, "test.csv"), sep='\t')
            train_contexts.extend(list(df.context.to_dense()))
            test_contexts.extend(list(df_tst.context.to_dense()))

../data/main/wiki-wiki
../data/main/active-dict
../data/main/bts-rnc


In [18]:
import pymystem3

In [19]:
stemmer = pymystem3.Mystem()

In [30]:
import re

pattern = re.compile(r'[^\W\d_]', re.U)

In [35]:
pattern.match("вапв")

In [39]:
text = ""

for tr, ts in zip(train_contexts, test_contexts):
    tr = stemmer.lemmatize(tr)
    ts = stemmer.lemmatize(ts)
    if len(tr)>1:
        text+=" ".join([a for a in tr if pattern.match(a)])
    text+='\n'
    if len(ts)>1:
        text+=" ".join([a for a in ts if pattern.match(a)])
    text+='\n'

In [40]:
len(text)

1680565

In [43]:
with open("../data/my_data/main_contexts_and_test.txt", 'w') as f:
    f.write(text)

# SenseGram

main file: main_wiki_and_contexts.txt

train senseGram:

        python train.py ~/Projects/russe_wsi/data/my_data/main_wiki_and_contexts.txt -cbow 0 -size 100 -window 3 -threads 10 -iter 2 -min_count 20 -only_letters -n 100 -N 200 -min_size 3

In [31]:
%load_ext autoreload
%autoreload 2

In [32]:
import sensegram

In [53]:
sv = sensegram.SenseGram.load_word2vec_format("../../sensegram/model/main_wiki_fixed.senses.w2v")

In [52]:
import codecs
with codecs.open("../../sensegram/model/main_wiki_and_contexts.txt.senses.w2v.probs",
                 "r",encoding='utf-8', errors='strict') as fdata,\
open("../../sensegram/model/main_wiki_fixed.senses.w2v.probs", 'w') as fout:
    lines = fdata.readlines()
    for l in lines:
        if l[0] == " ":
            l=l[1:]
        fout.write(l)

In [58]:
sv.get_senses("ключ")

[('ключ#0', 1.0)]

In [142]:
sv.most_similar("ключ#0")

/home/fogside/virtualenvs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('хеш#1', 0.9867193102836609),
 ('шифрование#0', 0.985775351524353),
 ('криптоаналитик#0', 0.9837445616722107),
 ('шифр#0', 0.9721437096595764),
 ('rsa#0', 0.9713340997695923),
 ('алгоритм#0', 0.9692728519439697),
 ('пароль#0', 0.9675806760787964),
 ('зашифровывать#0', 0.9658753275871277),
 ('криптографический#0', 0.9654587507247925),
 ('шифровать#0', 0.9600170850753784)]

In [56]:
sv.most_similar("девушка#1")

/home/fogside/virtualenvs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('девочка#0', 0.9784267544746399),
 ('подросток#0', 0.9768164157867432),
 ('мальчик#1', 0.9764024019241333),
 ('юноша#0', 0.9754001498222351),
 ('парень#0', 0.9505115747451782),
 ('женщина#0', 0.9427937865257263),
 ('детство#0', 0.9385959506034851),
 ('девчонка#0', 0.9384766817092896),
 ('малыш#0', 0.936809241771698),
 ('мужчина#0', 0.925966739654541)]

In [57]:
sv.most_similar("ключ#0")

/home/fogside/virtualenvs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('хеш#1', 0.9867193102836609),
 ('шифрование#0', 0.985775351524353),
 ('криптоаналитик#0', 0.9837445616722107),
 ('шифр#0', 0.9721437096595764),
 ('rsa#0', 0.9713340997695923),
 ('алгоритм#0', 0.9692728519439697),
 ('пароль#0', 0.9675806760787964),
 ('зашифровывать#0', 0.9658753275871277),
 ('криптографический#0', 0.9654587507247925),
 ('шифровать#0', 0.9600170850753784)]

In [138]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load_word2vec_format("../../sensegram/model/main_wiki_and_contexts.txt.words",\
                                            binary=True)

In [139]:
sv.get_senses("девушка")

[('девушка#0', 0.5), ('девушка#1', 0.5)]

In [146]:
wsd_model = sensegram.WSD(sv, wv, window=5, method='sim', filter_ctx=3)

Disambiguation method: sim
Filter context: f = 3


In [153]:
wsd_model.dis_text(' лес и птица петь', 'бор')[0]

'бор#0'

# AdaGram
./train_adagram.sh

    adapath="/home/fogside/.julia/v0.4/AdaGram"
    input_text_path="/home/fogside/Projects/russe_wsi/data/my_data/main_wiki_and_contexts.txt"
    python_ada_path="/home/fogside/Projects/python-adagram"
    dict_path="/home/fogside/Projects/russe_wsi/data/my_data/dict.txt"

    julia_model_path="/home/fogside/Projects/russe_wsi/adagram_model/adamodel_julia.model"
    python_model_path="/home/fogside/Projects/russe_wsi/adagram_model/adamodel_python.model"
    json_model_path="/home/fogside/Projects/russe_wsi/adagram_model/"

    echo "\nMAKING DICT\n"

    ${adapath}/utils/dictionary.sh ${input_text_path} ${dict_path}

    echo "\nSTART TRAIN\n"

    ${adapath}/train.sh --window 3 --workers 5 --min-freq 50 --remove-top-k 0 --dim 100 --prototypes 6 --alpha 0.1 --d 0 --epochs 2 --init-count 2 ${input_text_path} ${dict_path} ${julia_model_path}

    echo "\nCONVERT TO JSON\n"
    julia ${python_ada_path}/adagram/dump_julia.jl ${julia_model_path} ${json_model_path}

    echo "\nCONVERT TO pcl\n"
    python ${python_ada_path}/adagram/load_julia.py ${json_model_path} ${python_model_path}




In [66]:
import numpy as np

In [94]:
import adagram
vm = adagram.VectorModel.load('../adagram_model/adamodel_python.model')
vm.word_sense_probs('ключ')

[(0, 0.5371214112621437), (1, 0.4628586495898119)]

In [93]:
vm.sense_neighbors('ключ', 1)

/home/fogside/virtualenvs/py3/lib/python3.6/site-packages/adagram/model.py:124: RuntimeWarning: invalid value encountered in true_divide
  sim_matrix = np.dot(self.In, s_v) / self.InNorms


[('ворона', 1, 0.55260473),
 ('чулан', 0, 0.5376596),
 ('разгадка', 0, 0.5371159),
 ('палантир', 0, 0.5341252),
 ('песок', 2, 0.53401995),
 ('сундук', 0, 0.5311672),
 ('гек', 0, 0.5289355),
 ('дырка', 1, 0.51996446),
 ('сокровище', 0, 0.5175763),
 ('шкатулка', 0, 0.5140402)]

In [95]:
vm.sense_neighbors('ключ', 0)

/home/fogside/virtualenvs/py3/lib/python3.6/site-packages/adagram/model.py:124: RuntimeWarning: invalid value encountered in true_divide
  sim_matrix = np.dot(self.In, s_v) / self.InNorms


[('шифрование', 0, 0.8472483),
 ('пароль', 0, 0.8358505),
 ('шифровать', 0, 0.8323293),
 ('шифротекст', 0, 0.81765425),
 ('идентификатор', 0, 0.8144029),
 ('хеш', 0, 0.8107108),
 ('шифроваться', 0, 0.8012219),
 ('rsa', 0, 0.79929775),
 ('метка', 0, 0.7991079),
 ('бит', 0, 0.79688764)]

In [82]:
vm.sense_neighbors('суд', 1)

/home/fogside/virtualenvs/py3/lib/python3.6/site-packages/adagram/model.py:124: RuntimeWarning: invalid value encountered in true_divide
  sim_matrix = np.dot(self.In, s_v) / self.InNorms


[('ответчик', 0, 0.74774086),
 ('присяжный', 0, 0.7146408),
 ('подсудимый', 0, 0.7118493),
 ('приговор', 0, 0.7045118),
 ('обвиняемый', 0, 0.6943049),
 ('обжалование', 0, 0.67309785),
 ('обвинительный', 0, 0.6626315),
 ('апелляция', 0, 0.6526641),
 ('вердикт', 0, 0.6450886),
 ('оправдательный', 0, 0.64476454)]

In [72]:
vm.disambiguate('бор', ['химический', 'элемент'])

/home/fogside/virtualenvs/py3/lib/python3.6/site-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


array([0.00464293, 0.99435053, 0.00100655, 0.        , 0.        ,
       0.        ])

# Predict on main and test

## MAIN

In [14]:
import pandas as pd

In [106]:
from testing_interface import make_data,\
                              visualize_pca,\
                              evaluate_weighted_ari,\
                              get_one_word_data,\
                              visualize_pca_one_word,\
                              visualize_tsne_one_word

DATASET = '../data/main/wiki-wiki/train.csv'

contexts, main_words, labels, word_list_uniq = make_data(df_file_name=DATASET,\
                 use_gensim_simple_preproc=False, use_mystem=True,
                 throw_main_word=False, tokenize=True)

In [118]:
def check_vm(vm, DATASET):
    contexts, main_words, labels, word_list_uniq, positions = make_data(df_file_name=DATASET,\
                 use_gensim_simple_preproc=False, use_mystem=True,
                 throw_main_word=False, tokenize=True)
    preds = []
    words_pass = []
    for c,w in zip(contexts, main_words):
        try:
            if w not in words_pass:
                print("\n => ", w)
                print("Sense probs: ", vm.word_sense_probs(w))
                words_pass.append(w)
            pred = np.argmax(vm.disambiguate(w, c))
            preds.append(pred)
        except:
            if w not in words_pass:
                print("KEY_ERROR FOR WORD: ", w)
                words_pass.append(w)
            w_l = stemmer.lemmatize(w)[0]
            try:
                pred = np.argmax(vm.disambiguate(w_l, c))
                preds.append(pred)
            except:
                print("It doesnt work:(")
                preds.append(1)
    print("="*10)
    evaluate_weighted_ari(DATASET, preds)

In [119]:
check_vm(vm, DATASET)


 =>  замок
Sense probs:  [(0, 0.4294413337050045), (1, 0.570549595197531)]

 =>  лук
Sense probs:  [(0, 0.6774928605585753), (1, 0.3224625669335681)]

 =>  суда
KEY_ERROR FOR WORD:  суда

 =>  бор
Sense probs:  [(0, 0.5332279110980955), (1, 0.3319241595586044), (2, 0.13479602123569534)]
word	ari	count
бор	0.944344	56
замок	0.059148	138
лук	-0.016547	110
суда	0.000000	135
	0.134910	439


/home/fogside/virtualenvs/py3/lib/python3.6/site-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


In [116]:
check_vm(vm, "../data/main/active-dict/train.csv")


 =>  дар
Sense probs:  [(0, 0.7555856780107321), (1, 0.24437558654635302)]

 =>  двигатель
Sense probs:  [(0, 0.3831832371365644), (1, 0.3646499905319449), (2, 0.25215983038266365)]

 =>  двойник
Sense probs:  [(0, 0.9997724440826495)]

 =>  дворец
Sense probs:  [(0, 0.9999913674356269)]

 =>  девятка
Sense probs:  [(0, 0.9991635006840318)]

 =>  дедушка
Sense probs:  [(0, 0.9998501221716803)]

 =>  дежурная
KEY_ERROR FOR WORD:  дежурная
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(

 =>  дежурный
Sense probs:  [(0, 0.7423861262637023), (1, 0.25738061325815886)]

 =>  декабрист
Sense probs:  [(0, 0.9998755610191105)]

 =>  декрет
Sense probs:  [(0, 0.9999410682031118)]

 =>  дело
Sense probs:  [(0, 0.2591022178322957), (1, 0.4965760329676588), (2, 0.24431930776956728)]

 =>  демобилизация
Sense probs:  [(0, 0.9996692879294891)]



/home/fogside/virtualenvs/py3/lib/python3.6/site-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)



 =>  заведение
Sense probs:  [(0, 0.6778782991758439), (1, 0.32210539217060574)]

 =>  завещание
Sense probs:  [(0, 0.999933308142241)]

 =>  зависимость
Sense probs:  [(0, 0.6702027403836786), (1, 0.32978939929444856)]

 =>  заголовок
Sense probs:  [(0, 0.9999200328131959)]

 =>  заготовка
Sense probs:  [(0, 0.9999049652690006)]

 =>  задание
Sense probs:  [(0, 0.9999735035361976)]

 =>  задача
Sense probs:  [(0, 0.47039530263365037), (1, 0.529598867478354)]

 =>  задержка
Sense probs:  [(0, 0.47478124794540066), (1, 0.5251603038127102)]

 =>  зажигалка
Sense probs:  [(0, 0.9996381051042993)]

 =>  закон
Sense probs:  [(0, 0.175184376202555), (1, 0.15311213515542987), (2, 0.39435161326202794), (3, 0.27734837684758606)]

 =>  закрытие
Sense probs:  [(0, 0.9999616429716187)]

 =>  заложник
Sense probs:  [(0, 0.9760262188293797), (1, 0.023926198612400792)]

 =>  замена
Sense probs:  [(0, 0.613471955117694), (1, 0.3865089196189068)]

 =>  западня
Sense probs:  [(0, 0.9989276562456122)]



In [120]:
check_vm(vm, "../data/main/bts-rnc/train.csv")


 =>  балка
Sense probs:  [(0, 0.5418648952181123), (1, 0.4580672706711972)]

 =>  вид
Sense probs:  [(0, 0.31481022417797566), (1, 0.32098540065183917), (2, 0.36420292933968895)]

 =>  винт
Sense probs:  [(0, 0.8121776573388734), (1, 0.1877638360009462)]

 =>  горн
Sense probs:  [(0, 0.9997034291524226)]

 =>  губа
Sense probs:  [(0, 0.5539755735653393), (1, 0.2840875002775562), (2, 0.16188985568815417)]

 =>  жаба
Sense probs:  [(0, 0.9997773738195473)]

 =>  клетка
Sense probs:  [(0, 0.7927041336632679), (1, 0.20728689456837046)]

 =>  крыло
Sense probs:  [(0, 0.6765878423116545), (1, 0.3234002899409075)]

 =>  купюра
Sense probs:  [(0, 0.9998374849369409)]

 =>  курица
Sense probs:  [(0, 0.6284143286644007), (1, 0.37149460671423074)]


/home/fogside/virtualenvs/py3/lib/python3.6/site-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)



 =>  лавка
Sense probs:  [(0, 0.9779021435564381), (1, 0.02203410190170463)]

 =>  лайка
Sense probs:  [(0, 0.9995361571361945)]

 =>  лев
Sense probs:  [(0, 0.4606973832393846), (1, 0.5392907020048496)]

 =>  лира
Sense probs:  [(0, 0.3650310002370803), (1, 0.6348504923252325)]

 =>  мина
Sense probs:  [(0, 0.6146932348038836), (1, 0.38528138407883633)]

 =>  мишень
Sense probs:  [(0, 0.6079998444815953), (1, 0.3919012166858117)]

 =>  обед
Sense probs:  [(0, 0.9999067578126837)]

 =>  оклад
Sense probs:  [(0, 0.9998421592643392)]

 =>  опушка
Sense probs:  [(0, 0.999760126014854)]

 =>  полис
Sense probs:  [(0, 0.9998398397286348)]

 =>  пост
Sense probs:  [(0, 0.7844279812176713), (1, 0.16487440299249026), (2, 0.05069113901139639)]

 =>  поток
Sense probs:  [(0, 0.4523818997830328), (1, 0.3412299032587043), (2, 0.20637411068673064)]

 =>  проказа
Sense probs:  [(0, 0.9996374834859721)]

 =>  пропасть
Sense probs:  [(0, 0.9997881367597958)]

 =>  проспект
Sense probs:  [(0, 0.677628

In [156]:
def check_sense_gram(sg, DATASET):
    contexts, main_words, labels, word_list_uniq, positions = make_data(df_file_name=DATASET,\
                 use_gensim_simple_preproc=False, use_mystem=True,
                 throw_main_word=False, tokenize=False)
    preds = []
    words_pass = []
    
    for c,w in zip(contexts, main_words):
        try:
            if w not in words_pass:
                print("\n => ", w)
                print("Sense probs: ", vm.word_sense_probs(w))
                words_pass.append(w)
            pred = wsd_model.dis_text(c, w)[0]
            preds.append(pred)
        except:
            if w not in words_pass:
                print("KEY_ERROR FOR WORD: ", w)
                words_pass.append(w)
            w_l = stemmer.lemmatize(w)[0]
            try:
                pred = wsd_model.dis_text(c, w_l)[0]
                preds.append(pred)
            except:
                print("It doesnt work:(")
                preds.append('1')
    print("="*10)
    evaluate_weighted_ari(DATASET, preds)
    return preds

In [158]:
res = check_sense_gram(wsd_model, DATASET)


 =>  замок
Sense probs:  [(0, 0.4294413337050045), (1, 0.570549595197531)]
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doesnt work:(
It doe